# 技術メモ: Phase 1 — 根本的な環境構築の見直し（Clean Image Strategy）

## 1. 結論

- Paperspace標準の「PyTorchランタイム」ではなく、**「Start from Scratch」**を選択する。
- 対策として、Docker Hubの公式安定版イメージ **`pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel`** を直接指定する。

**イメージ選定の根拠:**

- **PyTorch 2.6.0:** CUDA 12.4環境で動作する最新安定版
- **cuda12.4:** Paperspaceホスト側のNVIDIAドライババージョン（12040）に適合し、起動時の互換性エラーを回避
- **devel:** ComfyUIのカスタムノード追加時（C++/CUDA拡張）に必須となるコンパイラを内包

これにより、システム標準ライブラリとの競合（ImportError）とドライババージョン不一致を完全に排除する。

## 2. 推奨される環境構成

Notebook作成時の設定を以下に変更する。

| 項目 | 設定値 |
|------|--------|
| **Runtime** | Start from Scratch |
| **Advanced Options > Container** | `pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel` |
| **Advanced Options > Command** | 下記コマンド（Jupyter本体はシステム側にインストール。例外的に許容される構成） |

**Command（コピー用）:**

```
apt-get update && apt-get install -y git python3-venv && pip install jupyterlab --break-system-packages && PIP_DISABLE_PIP_VERSION_CHECK=1 jupyter lab --allow-root --ip=0.0.0.0 --no-browser --ServerApp.trust_xheaders=True --ServerApp.disable_check_xsrf=False --ServerApp.allow_remote_access=True --ServerApp.allow_origin='*' --ServerApp.allow_credentials=True
```

## 3. クリーンインストール手順（Jupyter Notebook用）

Jupyter環境のセルに以下のコードを貼り付けて**順に実行**する。  
ディレクトリ移動には状態を維持する `%cd` を使用し、コマンド実行には `!` を使用する。

In [1]:
# 1. ワークスペースの準備
%cd /notebooks
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI

/notebooks
Cloning into 'ComfyUI'...
remote: Enumerating objects: 33022, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (125/125), done.
remote: Total 33022 (delta 108), reused 41 (delta 41), pack-reused 32856 (from 4)
Receiving objects: 100% (33022/33022), 78.61 MiB | 37.28 MiB/s, done.
Resolving deltas: 100% (22513/22513), done.
/notebooks/ComfyUI


In [2]:
# 2. 仮想環境（venv）の作成
# Ubuntu/Debian系イメージのため、事前に python3-venv がインストールされている必要がある
!python3 -m venv venv

In [3]:
# 3. 依存関係のインストール（仮想環境内のpipを直接指定）
# ホスト側のNVIDIAドライバ(12.4)に合わせたPyTorchをインストールする
!./venv/bin/pip install --upgrade pip
!./venv/bin/pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!./venv/bin/pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 550.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 156.7 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 88.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 114.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 109.0 MB/s  0:00:0700:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 113.8 MB/s  0:00:0200:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 40.9 MB/s  0:00:05:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 161.3 MB/s  0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 122.1 MB/s  0:00:0100:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
# 4. マネージャー導入
%cd custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ..

/notebooks/ComfyUI/custom_nodes
Cloning into 'ComfyUI-Manager'...
remote: Enumerating objects: 28581, done.
remote: Counting objects: 100% (951/951), done.
remote: Compressing objects: 100% (416/416), done.
remote: Total 28581 (delta 784), reused 538 (delta 535), pack-reused 27630 (from 3)
Receiving objects: 100% (28581/28581), 129.86 MiB | 52.13 MiB/s, done.
Resolving deltas: 100% (21200/21200), done.
/notebooks/ComfyUI


In [5]:
# 5. 起動とPublic URLの発行
# Paperspace固有の仕様を利用し、ポート6006で起動することでプロキシURLを生成する
import os
fqdn = os.environ.get('PAPERSPACE_FQDN', 'localhost')
print(f"\n🔗 ComfyUI Public URL: https://tensorboard-{fqdn}\n")

!./venv/bin/python main.py --listen 0.0.0.0 --port 6006

[START] Security scan
[DONE] Security scan
## ComfyUI-Manager: installing dependencies. (GitPython)
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2026-02-23 04:33:44.858
** Platform: Linux
** Python version: 3.11.11 | packaged by conda-forge | (main, Dec  5 2024, 14:17:24) [GCC 13.3.0]
** Python executable: /notebooks/ComfyUI/venv/bin/python
** ComfyUI Path: /notebooks/ComfyUI
** ComfyUI Base Folder Path: /notebooks/ComfyUI
** User directory: /notebooks/ComfyUI/user
** ComfyUI-Manager config path: /notebooks/ComfyUI/user/__manager/config.ini
** Log path: /notebooks/ComfyUI/user/comfyui.log

Prestartup times for custom nodes:
   9.0 seconds: /notebooks/ComfyUI/custom_nodes/ComfyUI-Manager

Found comfy_kitchen backend triton: {'available': True, 'disabled': True, 'unavailable_reason': None, 'capabilities': ['apply_rope', 'apply_rope1', 'dequantize_nvfp4', 'dequantize_per_tensor_fp8', 'quantize_nvfp4', 'quantize_per_tensor_fp8']}
Found comfy_kitchen backend cu

## 4. なぜこの方法が「推奨」なのか

| 観点 | 説明 |
|------|------|
| **システム保護** | `./venv/bin/pip` を使用することで、OS側のPythonパッケージ（Jupyter等）とComfyUI用のパッケージ（PyTorch等）が完全に分離される。 |
| **確実な実行** | パスを直接指定する方法は、Jupyterのマジックコマンドからでも確実に動作する。 |
| **ドライバ互換性** | Paperspaceホスト側のドライババージョン(12.4)に完全に適合し、起動エラーを回避できる。 |

## 5. 次のアクション

1. 現在エラーが出ているNotebookを**破棄（削除）**する。
2. 新規に**「Start from Scratch」**を選び、Containerに **`pytorch/pytorch:2.6.0-cuda12.4-cudnn9-devel`** を指定して作り直す。
3. 起動後、上記「3. クリーンインストール手順」を**再度実行**する。